In [14]:
%load_ext autoreload
%autoreload 2
import lilac as ll

ll.set_project_dir('./data')

ds = ll.get_dataset('local', 'OpenHermes-2.5')

# cluster_id has many more clusters than the category (12k vs 300).
CLUSTER_PATH = ('prompt__cluster', 'cluster_id')  # ('prompt__cluster', 'category_id')

# Only select the top-k clusters, sorted by size.
TOP_K_CLUSTERS = 2_000
# The number of rows to select inside each cluster.
NUM_ROWS_PER_CLUSTER = 1

# When we select from a cluster, we select.
CLUSTER_SORT_BY = ('prompt__cluster', 'cluster_membership_prob')
CLUSTER_SORT_ORDER = ll.SortOrder.DESC

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
groups = ds.select_groups(CLUSTER_PATH, sort_by=ll.GroupsSortBy.COUNT)
print('Number of clusters:', len(groups.counts))

Number of clusters: 12034


In [13]:
from tqdm import tqdm

topk_clusters = groups.counts[:TOP_K_CLUSTERS]

ds.remove_labels(f'keep-{TOP_K_CLUSTERS}-{NUM_ROWS_PER_CLUSTER}')
for value, count in tqdm(topk_clusters):
  ds.add_labels(
    f'keep-{TOP_K_CLUSTERS}-{NUM_ROWS_PER_CLUSTER}',
    filters=[(CLUSTER_PATH, 'equals', value)],
    sort_by=[CLUSTER_SORT_BY],
    sort_order=CLUSTER_SORT_ORDER,
    limit=NUM_ROWS_PER_CLUSTER,
  )

100%|██████████| 2000/2000 [08:07<00:00,  4.10it/s]
